In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import argparse
import os
import math
import random
import time
from typing import Tuple, List, Optional, Dict

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms
from torchvision.datasets.folder import default_loader
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt

from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import pandas as pd

FastKANModule = None
_fastkan_import_error = None
try:
    # najczęściej: pip install fast-kan
    from fastkan import FastKAN as FastKANModule
except Exception as e1:
    _fastkan_import_error = e1
    try:
        # alternatywna nazwa paczki
        from fast_kan import FastKAN as FastKANModule
        _fastkan_import_error = None
    except Exception as e2:
        _fastkan_import_error = (e1, e2)

# ====================================================
# Narzędzia
# ====================================================

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


def count_params(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters())


def to_numpy_img(batch_tensor: torch.Tensor) -> np.ndarray:
    x = batch_tensor.detach().cpu().numpy()
    x = np.clip(x, 0.0, 1.0)
    return x[:, 0, :, :]


def psnr_ssim_batch(x: torch.Tensor, y: torch.Tensor) -> Tuple[float, float]:
    x_np = to_numpy_img(x)
    y_np = to_numpy_img(y)
    ps, ss = [], []
    for i in range(x_np.shape[0]):
        ps.append(peak_signal_noise_ratio(x_np[i], y_np[i], data_range=1.0))
        try:
            ss.append(structural_similarity(x_np[i], y_np[i], data_range=1.0, channel_axis=None))
        except TypeError:
            ss.append(structural_similarity(x_np[i], y_np[i], data_range=1.0, multichannel=False))
    return float(np.mean(ps)), float(np.mean(ss))


# ====================================================
# SPLAJNY dla KAN
# ====================================================

def _linear_spline(x, grid, coeff):
    grid_min, grid_max = grid[0], grid[-1]
    G = grid.shape[0]
    t = (x - grid_min) / (grid_max - grid_min) * (G - 1)
    t = torch.clamp(t, 0, G - 1 - 1e-6)
    t_floor = torch.floor(t)
    frac = t - t_floor
    li = t_floor.long().unsqueeze(-1)
    ui = (li + 1).clamp(max=G - 1)
    c0 = torch.gather(coeff, -1, li).squeeze(-1)
    c1 = torch.gather(coeff, -1, ui).squeeze(-1)
    return c0 * (1.0 - frac) + c1 * frac


def _bezier_spline(x, grid, coeff):
    grid_min, grid_max = grid[0], grid[-1]
    t = torch.clamp((x - grid_min) / (grid_max - grid_min), 0.0, 1.0)
    res = coeff
    N = coeff.shape[-1]
    for k in range(1, N):
        res = res[..., :N - k] * (1 - t).unsqueeze(-1) + res[..., 1:N - k + 1] * t.unsqueeze(-1)
    return res[..., 0]


def _catmull_rom_spline(x, grid, coeff):
    grid_min, grid_max = grid[0], grid[-1]
    N = coeff.shape[-1]
    segs = N - 3
    t_norm = torch.clamp((x - grid_min) / (grid_max - grid_min) * segs, 0, segs - 1e-6)
    i = torch.floor(t_norm).to(torch.int64)
    t_loc = (t_norm - i.to(x.dtype))
    idx = torch.stack([i, (i + 1).clamp(max=N - 1), (i + 2).clamp(max=N - 1), (i + 3).clamp(max=N - 1)], dim=-1)
    P = torch.gather(coeff, -1, idx)
    t = t_loc
    t2, t3 = t * t, t * t * t
    return 0.5 * (
        2 * P[..., 1] +
        (-P[..., 0] + P[..., 2]) * t +
        (2 * P[..., 0] - 5 * P[..., 1] + 4 * P[..., 2] - P[..., 3]) * t2 +
        (-P[..., 0] + 3 * P[..., 1] - 3 * P[..., 2] + P[..., 3]) * t3
    )


def _bspline_spline(x, grid, coeff):
    grid_min, grid_max = grid[0], grid[-1]
    N = coeff.shape[-1]
    segs = N - 3
    t_norm = torch.clamp((x - grid_min) / (grid_max - grid_min) * segs, 0, segs - 1e-6)
    i = torch.floor(t_norm).to(torch.int64)
    t_loc = t_norm - i.to(x.dtype)
    idx = torch.stack([i, (i + 1).clamp(max=N - 1), (i + 2).clamp(max=N - 1), (i + 3).clamp(max=N - 1)], dim=-1)
    P = torch.gather(coeff, -1, idx)
    t = t_loc
    t2, t3 = t * t, t * t * t
    B0 = (1 - t) ** 3 / 6.0
    B1 = (3 * t3 - 6 * t2 + 4) / 6.0
    B2 = (-3 * t3 + 3 * t2 + 3 * t + 1) / 6.0
    B3 = t3 / 6.0
    return P[..., 0] * B0 + P[..., 1] * B1 + P[..., 2] * B2 + P[..., 3] * B3


def spline_interpolate(x, grid, coeff, spline_type: str):
    if spline_type == "linear":
        return _linear_spline(x, grid, coeff)
    elif spline_type == "bezier":
        return _bezier_spline(x, grid, coeff)
    elif spline_type == "catmull_rom":
        return _catmull_rom_spline(x, grid, coeff)
    elif spline_type == "bspline":
        return _bspline_spline(x, grid, coeff)
    elif spline_type in ("hermite", "nurbs"):
        return _linear_spline(x, grid, coeff)
    else:
        raise ValueError(f"Unknown spline: {spline_type}")


# ====================================================
# KAN 
# ====================================================

class KANLayer(nn.Module):
    def __init__(self, n_in: int, n_out: int, num_grid: int = 6, grid_min: float = -5.0, grid_max: float = 5.0,
                 spline_type: str = "bspline"):
        super().__init__()
        self.n_in, self.n_out, self.spline_type = n_in, n_out, spline_type
        grid = torch.linspace(grid_min, grid_max, steps=num_grid)
        self.register_buffer("grid", grid)
        self.coeffs = nn.Parameter(torch.randn(n_out, n_in, num_grid) * 0.05)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B = x.size(0)
        x_e = x.unsqueeze(1).expand(B, self.n_out, self.n_in)
        coeff_e = self.coeffs.unsqueeze(0).expand(B, -1, -1, -1)
        sp = spline_interpolate(x_e, self.grid, coeff_e, spline_type=self.spline_type)
        y = sp.sum(dim=-1)
        return y


class KANAutoencoder(nn.Module):
    def __init__(self, H: int, W: int, latent_dim: int = 256, num_grid: int = 6, spline_type: str = "bspline"):
        super().__init__()
        self.H, self.W, self.latent_dim = H, W, latent_dim
        self.input_dim = H * W
        self.enc = KANLayer(self.input_dim, latent_dim, num_grid=num_grid, spline_type=spline_type)
        self.dec = nn.Sequential(
            KANLayer(latent_dim, self.input_dim, num_grid=num_grid, spline_type=spline_type),
            nn.Sigmoid(),
        )

    def forward(self, x: torch.Tensor):
        B = x.size(0)
        flat = x.view(B, -1)
        z = self.enc(flat)
        recon_flat = self.dec(z)
        recon = recon_flat.view(B, 1, self.H, self.W)
        return recon, z


# ====================================================
# FastKAN
# ====================================================

class FastKANAutoencoder(nn.Module):
    """
    Autoenkoder oparty o FastKAN: dwie sieci FastKAN (encoder i decoder).
    Wymaga zainstalowanej biblioteki fast-kan / fastkan. Jeśli brak, rzuci czytelny błąd przy inicjalizacji.
    """
    def __init__(self, H: int, W: int, latent_dim: int = 256):
        super().__init__()
        if FastKANModule is None:
            raise ImportError(
                "FastKAN nie jest zainstalowany. Zainstaluj: `pip install fast-kan` (lub `pip install fastkan`). "
                f"Szczegóły importu: {repr(_fastkan_import_error)}"
            )
        self.H, self.W = H, W
        input_dim = H * W
        # Prosta architektura: warstwa wej -> latent oraz latent -> wyjście.
        # (Biblioteka akceptuje listę rozmiarów warstw, podobnie jak w przykładzie z klasyfikacją.)
        self.enc = FastKANModule([input_dim, latent_dim])
        self.dec_core = FastKANModule([latent_dim, input_dim])
        self.dec_out = nn.Sigmoid()

    def forward(self, x: torch.Tensor):
        B = x.size(0)
        flat = x.view(B, -1)
        z = self.enc(flat)                    # [B, L]
        recon_flat = self.dec_out(self.dec_core(z))  # [B, H*W] -> [0,1]
        recon = recon_flat.view(B, 1, self.H, self.W)
        return recon, z


# ====================================================
# CNN
# ====================================================

class CNNAutoencoder(nn.Module):
    def __init__(self, H: int, W: int, latent_dim: int = 256, base_ch: int = 32):
        super().__init__()
        self.H, self.W, self.latent_dim = H, W, latent_dim
        min_hw = min(H, W)
        max_blocks = 4
        target_S = 8
        n_down = 0
        S = min_hw
        while S > target_S and n_down < max_blocks:
            S = (S + 1) // 2
            n_down += 1
        self.S = S

        # Encoder
        enc_layers: List[nn.Module] = []
        ch_in = 1
        ch = base_ch
        for _ in range(n_down):
            enc_layers += [
                nn.Conv2d(ch_in, ch, 3, 2, 1), nn.ReLU(inplace=True),
                nn.Conv2d(ch, ch, 3, 1, 1), nn.ReLU(inplace=True),
            ]
            ch_in = ch
            ch = min(4 * base_ch, ch * 2)
        self.encoder_cnn = nn.Sequential(*enc_layers) if enc_layers else nn.Identity()
        self.encoder_gap = nn.AdaptiveAvgPool2d(1)
        self.encoder_fc = nn.Linear(ch_in, latent_dim)

        # Decoder
        dec_ch = ch_in
        self.decoder_fc = nn.Linear(latent_dim, dec_ch * self.S * self.S)
        dec_layers: List[nn.Module] = []
        ch_in = dec_ch
        for _ in range(n_down):
            out_c = ch_in // 2 if ch_in > base_ch else base_ch
            dec_layers += [
                nn.ConvTranspose2d(ch_in, out_c, 3, 2, 1, output_padding=1), nn.ReLU(inplace=True),
                nn.Conv2d(out_c, out_c, 3, 1, 1), nn.ReLU(inplace=True),
            ]
            ch_in = out_c
        self.decoder_cnn = nn.Sequential(*dec_layers) if dec_layers else nn.Identity()
        self.decoder_out = nn.Sequential(nn.Conv2d(ch_in, 1, 3, 1, 1), nn.Sigmoid())

    def forward(self, x: torch.Tensor):
        h = self.encoder_cnn(x)
        gap = self.encoder_gap(h).flatten(1)
        z = self.encoder_fc(gap)
        d = self.decoder_fc(z).view(z.size(0), -1, self.S, self.S)
        d = self.decoder_cnn(d)
        recon = self.decoder_out(d)
        if recon.size(2) != self.H or recon.size(3) != self.W:
            recon = F.interpolate(recon, size=(self.H, self.W), mode="bilinear", align_corners=False)
        return recon, z


# ====================================================
# Zbiory danych (grayscale) + wrapper AEFromTorchvision
# ====================================================

class AEFromTorchvision(Dataset):
    """Owija dataset torchvision (x, label) -> (x, x) dla autoenkodera."""
    def __init__(self, base_ds: Dataset):
        super().__init__()
        self.base = base_ds
    def __len__(self):
        return len(self.base)
    def __getitem__(self, idx):
        item = self.base[idx]
        if isinstance(item, tuple) and len(item) >= 1:
            x = item[0]
        else:
            x = item
        return x, x


class ShapesDataset(Dataset):
    def __init__(self, n_samples: int = 1000, H: int = 64, W: int = 64, seed: int = 123):
        super().__init__()
        self.n = n_samples
        self.H, self.W = H, W
        rng = np.random.RandomState(seed)
        self.params = []
        for _ in range(n_samples):
            shape = rng.choice(["circle", "square", "triangle", "line"])
            cx, cy = rng.randint(8, W - 8), rng.randint(8, H - 8)
            size = rng.randint(6, max(7, min(H, W) // 3))
            angle = float(rng.rand() * 360.0)
            self.params.append((shape, cx, cy, size, angle))
    def __len__(self):
        return self.n
    def __getitem__(self, idx):
        import PIL.Image as Image
        import PIL.ImageDraw as ImageDraw
        shape, cx, cy, size, angle = self.params[idx]
        img = Image.new("L", (self.W, self.H), color=0)
        draw = ImageDraw.Draw(img)
        if shape == "circle":
            bbox = (cx - size, cy - size, cx + size, cy + size)
            draw.ellipse(bbox, fill=255, outline=255)
        elif shape == "square":
            bbox = (cx - size, cy - size, cx + size, cy + size)
            draw.rectangle(bbox, fill=255, outline=255)
        elif shape == "triangle":
            p1 = (cx, cy - size)
            p2 = (cx - size, cy + size)
            p3 = (cx + size, cy + size)
            ang = math.radians(angle)
            def rot(p):
                x, y = p
                xr = cx + (x - cx) * math.cos(ang) - (y - cy) * math.sin(ang)
                yr = cy + (x - cx) * math.sin(ang) + (y - cy) * math.cos(ang)
                return (xr, yr)
            poly = list(map(rot, [p1, p2, p3]))
            draw.polygon(poly, fill=255, outline=255)
        elif shape == "line":
            length = size * 2
            x1, y1 = cx - length // 2, cy
            x2, y2 = cx + length // 2, cy
            ang = math.radians(angle)
            def rot2(x, y):
                xr = cx + (x - cx) * math.cos(ang) - (y - cy) * math.sin(ang)
                yr = cy + (x - cx) * math.sin(ang) + (y - cy) * math.cos(ang)
                return (xr, yr)
            p1 = rot2(x1, y1); p2 = rot2(x2, y2)
            draw.line([p1, p2], fill=255, width=2)
        x = transforms.ToTensor()(img)
        return x, x

def plot_models_comparison_matplotlib(orig5: torch.Tensor,
                                      recons_by_model: Dict[str, torch.Tensor],
                                      order: List[str],
                                      title: str,
                                      out_png: str,
                                      show: bool = True,
                                      dpi: int = 120) -> None:
    """
    Rysuje planszę porównawczą w Matplotlib z podpisami:
    wiersz 0: 'original', kolejne: nazwy modeli z 'order' (jeśli dostępne).
    Zapisuje do out_png i (opcjonalnie) pokazuje w Jupyterze.
    """
    os.makedirs(os.path.dirname(out_png), exist_ok=True)
    # realna liczba wierszy (oryginał + te modele, które mamy)
    available = [m for m in order if m in recons_by_model]
    n_rows = 1 + len(available)
    n_cols = min(5, orig5.size(0))

    fig, axes = plt.subplots(n_rows, n_cols,
                             figsize=(n_cols * 2.4, n_rows * 2.4),
                             squeeze=False)

    # wiersz 0: oryginały
    for c in range(n_cols):
        img = orig5[c].detach().cpu().squeeze(0).numpy()
        ax = axes[0, c]
        ax.imshow(img, cmap="gray", vmin=0.0, vmax=1.0)
        ax.set_xticks([]); ax.set_yticks([])
        if c == 0:
            ax.set_ylabel("original", fontsize=10)
        ax.set_title(f"img {c+1}", fontsize=9)

    # kolejne wiersze: modele
    for r, name in enumerate(available, start=1):
        recs = recons_by_model[name]
        for c in range(n_cols):
            img = recs[c].detach().cpu().squeeze(0).numpy()
            ax = axes[r, c]
            ax.imshow(img, cmap="gray", vmin=0.0, vmax=1.0)
            ax.set_xticks([]); ax.set_yticks([])
            if c == 0:
                ax.set_ylabel(name, fontsize=10)

    if title:
        fig.suptitle(title, fontsize=12, y=0.995)
    plt.tight_layout()
    fig.savefig(out_png, bbox_inches="tight", dpi=dpi)
    if show:
        plt.show()
    plt.close(fig)

def get_loaders(dataset_name: str,
                data_root: str,
                batch_small: int = 64,
                batch_large: int = 16,
                large_resize_to: int = 64,
                custom_large_dir: Optional[str] = None) -> Tuple[DataLoader, DataLoader, Tuple[int, int]]:

    if dataset_name == "mnist":
        H, W = 28, 28
        tfm = transforms.ToTensor()
        
        g = torch.Generator().manual_seed(42)
        full_train = datasets.MNIST(root=data_root, train=True, download=True, transform=tfm)
        
        val_size = int(0.2 * len(full_train))
        train_size = len(full_train) - val_size
        base_train, base_test = random_split(full_train, [train_size, val_size], generator=g)

        train = AEFromTorchvision(base_train)
        test  = AEFromTorchvision(base_test)
        return (DataLoader(train, batch_size=batch_small, shuffle=True, num_workers=0),
                DataLoader(test,  batch_size=batch_small, shuffle=False, num_workers=0),
                (H, W))

    elif dataset_name == "cifar16":
        H, W = 16, 16
        tfm = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize((H, W), interpolation=transforms.InterpolationMode.BICUBIC),
            transforms.ToTensor(),
        ])

        g = torch.Generator().manual_seed(42)
        full_train = datasets.CIFAR10(root=data_root, train=True, download=True, transform=tfm)

        val_size = int(0.2 * len(full_train))
        train_size = len(full_train) - val_size
        base_train, base_test = random_split(full_train, [train_size, val_size], generator=g)

        train = AEFromTorchvision(base_train)
        test  = AEFromTorchvision(base_test)
        return (DataLoader(train, batch_size=batch_small, shuffle=True, num_workers=0),
                DataLoader(test,  batch_size=batch_small, shuffle=False, num_workers=0),
                (H, W))

    elif dataset_name == "large":
        H = W = int(large_resize_to)
        if custom_large_dir and os.path.isdir(custom_large_dir):
            class FolderGray(Dataset):
                def __init__(self, root_dir: str, split: str = "train", val_ratio: float = 0.2):
                    files = []
                    for dirpath, _, filenames in os.walk(root_dir):
                        for fn in filenames:
                            if fn.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".webp", ".tif", ".tiff")):
                                files.append(os.path.join(dirpath, fn))
                    files.sort()
                    n = len(files)
                    n_val = max(1, int(n * val_ratio))
                    self.files = files[:-n_val] if split == "train" else files[-n_val:]
                    self.tfm = transforms.Compose([
                        transforms.Grayscale(num_output_channels=1),
                        transforms.Resize((H, W), interpolation=transforms.InterpolationMode.BICUBIC),
                        transforms.ToTensor(),
                    ])
                def __len__(self): return len(self.files)
                def __getitem__(self, idx):
                    img = default_loader(self.files[idx])
                    x = self.tfm(img)
                    return x, x
            base_train = FolderGray(custom_large_dir, split="train")
            base_test  = FolderGray(custom_large_dir, split="val")
        else:
            tfm = transforms.Compose([
                transforms.Grayscale(num_output_channels=1),
                transforms.Resize((H, W), interpolation=transforms.InterpolationMode.BICUBIC),
                transforms.ToTensor(),
            ])
            g = torch.Generator().manual_seed(42)
            full_train = datasets.STL10(root=data_root, download=True, transform=tfm)
    
            val_size = int(0.2 * len(full_train))
            train_size = len(full_train) - val_size
            base_train, base_test = random_split(full_train, [train_size, val_size], generator=g)
        train = AEFromTorchvision(base_train)
        test  = AEFromTorchvision(base_test)
        return (DataLoader(train, batch_size=batch_large, shuffle=True, num_workers=0),
                DataLoader(test,  batch_size=batch_large, shuffle=False, num_workers=0),
                (H, W))

    elif dataset_name == "shapes":
        H, W = 32, 32
        train = ShapesDataset(n_samples=800, H=H, W=W, seed=42)
        test  = ShapesDataset(n_samples=200, H=H, W=W, seed=42)
        return (DataLoader(train, batch_size=batch_small, shuffle=True, num_workers=0),
                DataLoader(test,  batch_size=batch_small, shuffle=False, num_workers=0),
                (H, W))

    else:
        raise ValueError(f"Nieznany zbiór: {dataset_name}")


# ====================================================
# Trening / Ewaluacja
# ====================================================

@torch.no_grad()
def evaluate(model: nn.Module, loader: DataLoader, device: torch.device, max_batches: Optional[int] = None):
    model.eval()
    total_loss, n_seen = 0.0, 0
    psnrs, ssims = [], []
    for b_idx, (x, y) in enumerate(loader):
        x = x.to(device); y = y.to(device)
        recon, _ = model(x)
        total_loss += F.mse_loss(recon, y, reduction="sum").item()
        n_seen += x.size(0)
        p, s = psnr_ssim_batch(y, recon)
        psnrs.append(p); ssims.append(s)
        if max_batches is not None and (b_idx + 1) >= max_batches:
            break
    return total_loss / max(1, n_seen), float(np.mean(psnrs)), float(np.mean(ssims))


def train_one_epoch(model: nn.Module, loader: DataLoader, opt: torch.optim.Optimizer, device: torch.device):
    model.train()
    running = 0.0
    for x, y in loader:
        x = x.to(device); y = y.to(device)
        opt.zero_grad(set_to_none=True)
        recon, _ = model(x)
        loss = F.mse_loss(recon, y)
        loss.backward(); opt.step()
        running += loss.item() * x.size(0)
    return running / len(loader.dataset)


@torch.no_grad()
def measure_inference_time(model: nn.Module, loader: DataLoader, device: torch.device, warmup: int = 2, measure_batches: int = 5) -> float:
    model.eval()
    it = iter(loader)
    for _ in range(warmup):
        try:
            x, _ = next(it)
        except StopIteration:
            it = iter(loader); x, _ = next(it)
        x = x.to(device); _ = model(x)
    times = []
    it = iter(loader)
    for _ in range(measure_batches):
        try:
            x, _ = next(it)
        except StopIteration:
            it = iter(loader); x, _ = next(it)
        x = x.to(device)
        t0 = time.perf_counter(); _ = model(x); t1 = time.perf_counter()
        times.append((t1 - t0) / x.size(0))
    return float(np.mean(times))


# ====================================================
# Budowa modeli + zapis porównań obrazków
# ====================================================

def build_model(kind: str, H: int, W: int, latent_dim: int, kan_grid: int) -> nn.Module:
    if kind == "cnn":
        return CNNAutoencoder(H, W, latent_dim=latent_dim, base_ch=32)
    elif kind.startswith("kan_"):
        spline_map = {"kan_linear": "linear", "kan_bezier": "bezier", "kan_catmull": "catmull_rom", "kan_bspline": "bspline"}
        return KANAutoencoder(H, W, latent_dim=latent_dim, num_grid=kan_grid, spline_type=spline_map[kind])
    elif kind == "fastkan":
        return FastKANAutoencoder(H, W, latent_dim=latent_dim)
    else:
        raise ValueError(f"Nieznany model: {kind}")


def save_sample_recons(model: nn.Module, loader: DataLoader, device: torch.device, out_path: str, n_batches: int = 1):
    model.eval(); os.makedirs(os.path.dirname(out_path), exist_ok=True)
    imgs = []
    with torch.no_grad():
        it = iter(loader)
        for _ in range(n_batches):
            try: x, _ = next(it)
            except StopIteration: break
            x = x.to(device); recon, _ = model(x)
            grid = make_grid(torch.cat([x[:8], recon[:8]], dim=0), nrow=8)
            imgs.append(grid.cpu())
    if imgs: save_image(torch.stack(imgs).mean(dim=0), out_path)


def save_models_comparison(orig5: torch.Tensor,
                           recons_by_model: Dict[str, torch.Tensor],
                           order: List[str],
                           out_png: str,
                           legend_txt: str):
    """
    Tworzy wspólną planszę: pierwszy wiersz to oryginały (5 obrazków),
    kolejne wiersze to rekonstrukcje wg kolejności w 'order' (jeśli dostępne).
    Zapisuje PNG + plik z legendą.
    """
    os.makedirs(os.path.dirname(out_png), exist_ok=True)
    rows = []
    rows.append(make_grid(orig5[:5].cpu(), nrow=5, padding=2))
    legend_lines = ["Row 0: original"]
    row_idx = 1
    for name in order:
        if name in recons_by_model:
            rows.append(make_grid(recons_by_model[name][:5].cpu(), nrow=5, padding=2))
            legend_lines.append(f"Row {row_idx}: {name}")
            row_idx += 1
    if not rows:
        return
    hcat = rows[0]
    for r in rows[1:]:
        if r.shape[2] != hcat.shape[2]:
            # wycentruj/pad do tej samej szerokości
            W = max(r.shape[2], hcat.shape[2])
            def _pad_to(t, W):
                pad = W - t.shape[2]
                if pad <= 0: return t[:, :, :W]
                left = pad // 2; right = pad - left
                return F.pad(t, (left, right, 0, 0))
            hcat = _pad_to(hcat, W)
            r = _pad_to(r, W)
        hcat = torch.cat([hcat, r], dim=1)  # po wysokości
    save_image(hcat, out_png)
    with open(legend_txt, "w", encoding="utf-8") as f:
        f.write("\n".join(legend_lines))

def run_all(datasets: List[str] = None,
            models: List[str] = None,
            data_root: str = "./data",
            latent_dim: int = 256,
            kan_grid: int = 6,
            epochs_small: int = 10,
            epochs_large: int = 8,
            batch_small: int = 64,
            batch_large: int = 16,
            large_resize: int = 64,
            large_dir: Optional[str] = None,
            lr: float = 1e-3,
            max_eval_batches: Optional[int] = None,
            save_samples: bool = False,
            outdir: str = "./outputs",
            csv_path: str = "./outputs/results_equal_latent.csv",
            seed: int = 42) -> pd.DataFrame:

    set_seed(seed); device = torch.device("cpu"); os.makedirs(outdir, exist_ok=True)
    if datasets is None: datasets = ["mnist", "cifar16", "large", "shapes"]
    if models is None: models = ["cnn", "kan_linear", "kan_bezier", "kan_catmull", "kan_bspline", "fastkan"]

    print(models)

    results: List[Dict] = []

    for ds in datasets:
        train_loader, test_loader, (H, W) = get_loaders(
            ds, data_root,
            batch_small=batch_small,
            batch_large=batch_large,
            large_resize_to=large_resize,
            custom_large_dir=large_dir,
        )
        epochs = epochs_small if ds in ("mnist", "cifar16", "shapes") else epochs_large

        # 5 obrazów do porównań między modelami
        cmp_batch = next(iter(test_loader))[0][:5].to(device)  # [5,1,H,W]
        recons_by_model: Dict[str, torch.Tensor] = {}

        for kind in models:
            model = build_model(kind, H, W, latent_dim=latent_dim, kan_grid=kan_grid)
            model.to(device)
            n_params = count_params(model)
            opt = torch.optim.Adam(model.parameters(), lr=lr)
            best_val = float("inf"); best_state = None

            for ep in range(1, epochs + 1):
                tr_loss = train_one_epoch(model, train_loader, opt, device)
                val_loss, val_psnr, val_ssim = evaluate(model, test_loader, device, max_batches=max_eval_batches)
                if val_loss < best_val:
                    best_val = val_loss
                    best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
                print(f"[{ds}][{kind}] ep {ep:03d}/{epochs} | train MSE {tr_loss:.5f} | val MSE {val_loss:.5f} | PSNR {val_psnr:.2f} dB | SSIM {val_ssim:.3f}")

            if best_state is not None: model.load_state_dict(best_state)
            val_loss, val_psnr, val_ssim = evaluate(model, test_loader, device, max_batches=max_eval_batches)
            sec_per_img = measure_inference_time(model, test_loader, device, warmup=2, measure_batches=5)

            # Zapisywanie próbki
            if save_samples:
                out_png = os.path.join(outdir, f"samples_{ds}_{kind}.png")
                save_sample_recons(model, test_loader, device, out_png, n_batches=1)

            # dodawanie rekonstrukcje do planszy
            with torch.no_grad():
                recon, _ = model(cmp_batch)
            recons_by_model[kind] = recon.detach().cpu()

            results.append({
                "dataset": ds,
                "H": H, "W": W,
                "model": kind,
                "latent_dim": latent_dim,
                "params": n_params,
                "epochs": epochs,
                "val_mse": val_loss,
                "val_psnr": val_psnr,
                "val_ssim": val_ssim,
                "infer_ms_per_img": sec_per_img * 1000.0,
            })
            del model

        # po przejściu wszystkich modeli dla danego zbioru — zapisz wspólną planszę porównawczą
        compare_png = os.path.join(outdir, f"compare_{ds}.png")
        compare_legend = os.path.join(outdir, f"compare_{ds}.txt")
        save_models_comparison(cmp_batch.cpu(), recons_by_model, models, compare_png, compare_legend)
        print(f"== Zapisano porównanie obrazów do: {compare_png} (legenda: {compare_legend})")

        # NOWE: Matplotlib z etykietami (pokazuje w Jupyterze + zapisuje)
        compare_mpl = os.path.join(outdir, f"compare_{ds}_mpl.png")
        plot_models_comparison_matplotlib(
            orig5=cmp_batch.cpu(),
            recons_by_model=recons_by_model,
            order=models,
            title=f"Dataset: {ds}  |  models: {', '.join([m for m in models if m in recons_by_model])}",
            out_png=compare_mpl,
            show=True,        # w Jupyterze wyświetli od razu
            dpi=120
        )
        print(f"== Zapisano porównanie Matplotlib do: {compare_mpl}")

    df = pd.DataFrame(results)
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    df.to_csv(csv_path, index=False)
    print("\n== Zapisano wyniki do:", csv_path)
    print(df)
    return df


def parse_args_jupyter_friendly():
    parser = argparse.ArgumentParser(description="Porównanie AE: CNN, KAN (splajny), FastKAN — wspólny latent")
    parser.add_argument("--data-root", type=str, default="./data")
    parser.add_argument("--datasets", type=str, default="mnist,cifar16,large,shapes")
    parser.add_argument("--models", type=str, default="cnn,kan_linear,kan_bezier,kan_catmull,kan_bspline,fastkan")
    parser.add_argument("--latent-dim", type=int, default=16)
    parser.add_argument("--kan-grid", type=int, default=6)
    parser.add_argument("--epochs-small", type=int, default=10)
    parser.add_argument("--epochs-large", type=int, default=10)
    parser.add_argument("--batch-small", type=int, default=64)
    parser.add_argument("--batch-large", type=int, default=16)
    parser.add_argument("--large-resize", type=int, default=64)
    parser.add_argument("--large-dir", type=str, default=None)
    parser.add_argument("--lr", type=float, default=1e-3)
    parser.add_argument("--max-eval-batches", type=int, default=None)
    parser.add_argument("--save-samples", action="store_true")
    parser.add_argument("--outdir", type=str, default="./outputs")
    parser.add_argument("--csv", type=str, default="./outputs/results_equal_latent_16_no_silu_test.csv")
    parser.add_argument("--seed", type=int, default=42)
    args, _unknown = parser.parse_known_args()
    return args


latens = [256, 128, 64, 32, 16];

for lat in latens:
    run_all(
        datasets=["mnist","cifar16","large","shapes"],
        models=["cnn","kan_linear","kan_bezier","kan_catmull","kan_bspline","fastkan"],
        data_root="./data",
        latent_dim=lat,
        kan_grid=6,
        epochs_small=10,
        epochs_large=10,
        batch_small=64,
        batch_large=64,
        large_resize=64,
        large_dir=None,
        lr=1e-3,
        max_eval_batches=None,
        save_samples="store_true",
        outdir="./outputs",
        csv_path=f"./outputs/results_final_{lat}.csv",
        seed=42,
    )


/opt/anaconda3/envs/OE/lib/python3.12/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/OE/lib/python3.12/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <367D4265-B20F-34BD-94EB-4F3EE47C385B> /opt/anaconda3/envs/OE/lib/python3.12/site-packages/torchvision/image.so
  Reason: tried: '/Users/sylwesterwieczorek/VulkanSDK/macOS/lib/libjpeg.9.dylib' (no such file), '/libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/OE/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/OE/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/OE/lib/python3.12/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/OE/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. O

['cnn', 'kan_linear', 'kan_bezier', 'kan_catmull', 'kan_bspline', 'fastkan']
[mnist][cnn] ep 001/10 | train MSE 0.04954 | val MSE 18.78598 | PSNR 16.91 dB | SSIM 0.746
[mnist][cnn] ep 002/10 | train MSE 0.01805 | val MSE 11.16709 | PSNR 19.25 dB | SSIM 0.849
[mnist][cnn] ep 003/10 | train MSE 0.01275 | val MSE 9.05995 | PSNR 20.08 dB | SSIM 0.876
[mnist][cnn] ep 004/10 | train MSE 0.01044 | val MSE 7.67160 | PSNR 20.81 dB | SSIM 0.896
[mnist][cnn] ep 005/10 | train MSE 0.00893 | val MSE 6.62932 | PSNR 21.55 dB | SSIM 0.910
[mnist][cnn] ep 006/10 | train MSE 0.00799 | val MSE 5.96215 | PSNR 21.98 dB | SSIM 0.919
[mnist][cnn] ep 007/10 | train MSE 0.00730 | val MSE 5.67616 | PSNR 22.14 dB | SSIM 0.919
[mnist][cnn] ep 008/10 | train MSE 0.00677 | val MSE 5.30557 | PSNR 22.47 dB | SSIM 0.926
[mnist][cnn] ep 009/10 | train MSE 0.00633 | val MSE 4.96060 | PSNR 22.73 dB | SSIM 0.931
[mnist][cnn] ep 010/10 | train MSE 0.00597 | val MSE 4.72834 | PSNR 22.97 dB | SSIM 0.934
[mnist][kan_linear] e